# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [32]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [33]:
orders = pd.read_csv(r'Orders/Orders.csv')

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [34]:
print(orders.head())

orders['amount_spent'] = pd.to_numeric(orders['amount_spent'], errors='coerce')

customer_spend = orders.groupby('CustomerID')['amount_spent'].sum()




   Unnamed: 0  InvoiceNo StockCode  year  month  day  hour  \
0           0     536365    85123A  2010     12    3     8   
1           1     536365     71053  2010     12    3     8   
2           2     536365    84406B  2010     12    3     8   
3           3     536365    84029G  2010     12    3     8   
4           4     536365    84029E  2010     12    3     8   

                           Description  Quantity          InvoiceDate  \
0   white hanging heart t-light holder         6  2010-12-01 08:26:00   
1                  white metal lantern         6  2010-12-01 08:26:00   
2       cream cupid hearts coat hanger         8  2010-12-01 08:26:00   
3  knitted union flag hot water bottle         6  2010-12-01 08:26:00   
4       red woolly hottie white heart.         6  2010-12-01 08:26:00   

   UnitPrice  CustomerID         Country  amount_spent  
0       2.55       17850  United Kingdom         15.30  
1       3.39       17850  United Kingdom         20.34  
2       2.75     

In [35]:
vip_threshold = customer_spend.quantile(0.95)
preferred_threshold = customer_spend.quantile(0.75)

customer_df = customer_spend.to_frame(name= 'total_spent')
customer_df['customer_type'] = np.where(
    customer_df['total_spent'] > vip_threshold, 'VIP', 
    np.where(customer_df['total_spent'] > preferred_threshold, 'Preferred', 'Regular'))

print(customer_spend.head())

CustomerID
12346    77183.60
12347     4310.00
12348     1797.24
12349     1757.55
12350      334.40
Name: amount_spent, dtype: float64


In [36]:
orders_with_labels = orders.merge(customer_df[['customer_type']], on='CustomerID', how='left')

print(orders_with_labels[['CustomerID', 'amount_spent', 'customer_type']].head())


   CustomerID  amount_spent customer_type
0       17850         15.30     Preferred
1       17850         20.34     Preferred
2       17850         22.00     Preferred
3       17850         20.34     Preferred
4       17850         20.34     Preferred


In [37]:
# biggest whales
print(customer_df.sort_values(by='total_spent', ascending=False).head(10))

            total_spent customer_type
CustomerID                           
14646         280206.02           VIP
18102         259657.30           VIP
17450         194550.79           VIP
16446         168472.50           VIP
14911         143825.06           VIP
12415         124914.53           VIP
14156         117379.63           VIP
17511          91062.38           VIP
16029          81024.84           VIP
12346          77183.60           VIP


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [38]:
customer_countries = orders[["CustomerID", "Country"]].drop_duplicates()

vip_lookup = customer_df.merge(
    customer_countries,
    on="CustomerID",
    how="left" 
)



In [39]:
vips_only = vip_lookup[vip_lookup['customer_type'] == 'VIP']

vip_counts = vips_only['Country'].value_counts()

print('Top 5 countries with the most VIPs:')
print(vip_counts.head())

Top 5 countries with the most VIPs:
Country
United Kingdom    177
Germany            10
France              9
Switzerland         3
Australia           2
Name: count, dtype: int64


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [40]:
preferred_plus_vips = vip_lookup[(vip_lookup['customer_type'] == 'Preferred') | (vip_lookup['customer_type'] == 'VIP')]

preferred_plus_vips_counts = preferred_plus_vips['Country'].value_counts()

print('Top 5 countries with the most VIPs:')
print(preferred_plus_vips_counts.head())

Top 5 countries with the most VIPs:
Country
United Kingdom    932
Germany            39
France             29
Belgium            12
Switzerland         9
Name: count, dtype: int64
